In [ ]:
import os
from mpi4py import MPI
import openpmd_api as io
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import torch
torch.__version__

In [ ]:
simdir = "/bigdata/hplsim/production/KHI_for_GB_MR/runs/004_KHI_withRad_randomInit_seed_12"
simdir = "/bigdata/hplsim/aipp/SC24_PIConGPU-Continual-Learning/03-30_learning-rate-scaling-with-ranks_chamfersdistance_fix-gpu-volume/24-nodes_full-picongpu-data/"

#simdir = "/bigdata/hplsim/production/KHI_for_GB_MR/runs/002_KHI_withRad_randomInit"
def loadRad(t, simdir=simdir):
    radFile = "/".join((simdir, "simOutput/radiationOpenPMD/e_radAmplitudes_{}.bp5"))
    radFile = radFile.format(t)
    rad = io.Series(radFile, io.Access.read_only)
    it = rad.iterations[t]
    #for i in it.meshes.items():
    #    print(i)
    dd = tuple( (it.meshes["DetectorDirection"][d][:, 0,0] for d in ("x", "y", "z") ) )
    df = it.meshes["DetectorFrequency"]["omega"][0, :, 0]
    amp = tuple( (tuple ((it.meshes["Amplitude"]["_".join((d, c))][:] for c in ("Re", "Im"))) for d in ("x", "y", "z") ))
    ampd = tuple( it.meshes["Amplitude_distributed"][d][:] for d in ("x", "y", "z") )
    rad.flush()
    dd = np.stack(dd)
    ampd = np.stack(ampd)
    amp = tuple( (ampx[0] + 1j*ampx[1]) for ampx in amp )
    amp = np.stack(amp)
    return dd, df, amp, ampd

def loadRadDDc(t, ddc):  
    radFile = "/".join((simdir, "simOutput/radiationOpenPMD/e_radAmplitudes_{}.bp5"))
    radFile = radFile.format(t)
    rad = io.Series(radFile, io.Access.read_only)
    it = rad.iterations[t]
    #for i in it.meshes.items():
    #    print(i)
    df = it.meshes["DetectorFrequency"]["omega"][0, :, 0]
    amp = tuple( (tuple ((it.meshes["Amplitude"]["_".join((d, c))][ddz, :] for c in ("Re", "Im"))) for d in ("x", "y", "z") ))
    ampd = tuple( it.meshes["Amplitude_distributed"][d][:, ddz, :] for d in ("x", "y", "z") )
    rad.flush()
    ampd = np.stack(ampd)
    amp = tuple( (ampx[0] + 1j*ampx[1]) for ampx in amp )
    amp = np.stack(amp)
    return df, amp, ampd

def intens(amp):
    return np.abs(np.sum(amp*amp, axis = 0))    

In [ ]:
t = 950
dd, df, amp, ampd = loadRad(t)
I = intens(amp)

In [ ]:
dd1, df1, amp1, ampd1 = loadRad(1000)

In [ ]:
amp01 = amp - amp1
I01 = intens(amp01)

In [ ]:
for d, det in zip(("x" ,"y", "z"), dd):
    plt.plot(det, label=d)
plt.legend()
plt.show()

In [ ]:
ddz = 4
phi = np.arctan(dd[0][:ddz]/dd[1][:ddz]) / np.pi *180
phi[-1] *= -1
phi

In [ ]:
plt.ylabel("DetectorDirection phi")
plt.xlabel("Frequency")
#plt.semilogx()
plt.imshow((np.log(I[:ddz,0:450,0])), aspect="auto")
plt.show()
plt.imshow(np.log(I01[:ddz,0:450,0]), extent=(0, 450, 90, 0), aspect="auto")

In [ ]:
for i in range(ddz):
    plt.plot(I01[i,200:450,0])
plt.semilogy()
plt.show()
ddc = 0
plt.plot(I[ddc,200:450,0])
plt.plot(np.mean(I01[:,200:450,0], axis=0))
plt.semilogy()
plt.show()

In [ ]:
Id = np.abs(np.sum(ampd*ampd, axis = 0))

In [ ]:
plt.ylabel("GPU Box")
plt.xlabel("Frequency")
plt.imshow(np.log(Id[:,ddc, 300:450]), extent=(300, 450, 0, 64), aspect="auto")

In [ ]:
for i in range(16):
    plt.plot(Id[i,ddc, 200:450])
plt.semilogy()
plt.show()

In [ ]:
partFile = "/bigdata/hplsim/production/KHI_for_GB_MR/runs/002_KHI_withRad_randomInit/simOutput/openPMD/simData_00{}.bp"
part = io.Series(partFile.format(t), io.Access_Type.read_only)

In [ ]:
partit = part.iterations[t]
extent = tuple(( partit.particles["e_all"].particle_patches["extent"][d].load() for d in ("x", "y", "z")))
offset = tuple(( partit.particles["e_all"].particle_patches["offset"][d].load() for d in ("x", "y", "z")))

part.flush()
extent = np.stack(extent)
offset = np.stack(offset)

In [ ]:
np.sum(Id[:, ddc, 200] > 10)

In [ ]:
tib = np.sum(Id[:,ddc, 300:450], axis=-1)
plt.scatter(offset[0], offset[1], c=np.log(tib))

In [ ]:
r_offset = offset * np.array( tuple( partit.get_attribute(v) for v in ("cell_width", "cell_height", "cell_depth") ))[:, np.newaxis]
r_offset = r_offset.T

In [ ]:
def ampdbs(hib):
    print(np.sum(hib))
    iteration = t
    phase_shift = np.exp(-1.j * df[np.newaxis, np.newaxis, :] * (iteration + np.dot(r_offset[hib], dd)[:, :, np.newaxis] / 1.0))
    #phase_shift = 1.
    #print(phase_shift.shape, ampd[:, hib, :, :].shape)
    ampdhib = np.sum(ampd[:, hib, :, :]/phase_shift, axis=1)
    Ihib = np.abs(np.sum(ampdhib*ampdhib, axis = 0))
    return Ihib

minOmega = 0
#plt.plot(ampdbs(np.logical_and(offset[1] > 300, offset[1] < 430))[ddc, minOmega:450], label="vort")
plt.plot(ampdbs(np.logical_and(offset[1] > 180, offset[1] < 300))[ddc, minOmega:450], label="central")
plt.plot(ampdbs(np.logical_or(offset[1] < 50, offset[1] > 430))[ddc, minOmega:450], label="split")
#plt.plot(ampdbs(np.full(offset[0].shape, True))[ddc, minOmega:450], label="all")
#plt.plot(I01[ddc, minOmega:450], label="total", linestyle='dashed')
#plt.plot(ampdbs(Id[:, ddc, 200] < 10)[ddc, minOmega:450], label="signal")
#plt.plot(ampdbs(Id[:, ddc, 200] > 10)[ddc, minOmega:450], label="flat")
plt.legend()
plt.semilogy()
plt.show()

In [ ]:
dfl = []
ampl = []
ampdl = []
for t in range(10, 1000):
    print(t, "...")
    dft, ampt, ampdt = loadRadDDc(t, ddc)
    dfl.append(dft)
    ampl.append(ampt)
    ampdl.append(ampdt)

In [ ]:
dfl = np.stack(dfl)
ampl = np.stack(ampl)
ampdl = np.stack(ampdl)

In [ ]:
ampdl.shape

In [ ]:
Il = intens(ampl.transpose((1,0,2,3)))
print(Il.shape)
T0 = 10
#Il0 = intens((ampl - ampl[T0]).transpose((1,0,2,3)))
Il0 = intens((ampl[1:] - ampl[:-1]).transpose((1,0,2,3)))


In [ ]:
plt.imshow(np.log(Il[:,200:450,0]), aspect="auto")
plt.show()
plt.imshow(np.log(Il0[T0:,200:450,0]), aspect="auto")

In [ ]:
def ampdlbs(hib, Tmin=None, Tmax=None):
    ampdhib = np.sum(ampdl[Tmin:Tmax, :, hib, :], axis=(0,2))
    Ihib = np.abs(np.sum(ampdhib*ampdhib, axis = 0))
    return Ihib

In [ ]:
T = (T0,T0+100)
plt.plot(ampdlbs(np.full(offset[0].shape, True), *T)[minOmega:450], label="all")
plt.plot(Il0[-1, minOmega:450], label="total", linestyle='dashed')
plt.plot(ampdlbs(Id[:, ddc, 200] < 10, *T)[minOmega:450], label="signal")
plt.plot(ampdlbs(Id[:, ddc, 200] > 10, *T)[minOmega:450], label="flat")
plt.legend()
plt.semilogy()
plt.show()

In [ ]:
Id.shape

In [ ]:
!ls /bigdata/hplsim/production/KHI_for_GB_MR/runs/
